In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import pickle

In [25]:
fpath = 'C:/Users/jbachlombardo/Documents/Community Colleges/Data/College scorecard/CollegeScorecard_Raw_Data/'

In [26]:
fifteen = 'MERGED2001_02_PP.csv'
ten = 'MERGED2006_07_PP.csv'
five = 'MERGED2011_12_PP.csv'
current = 'MERGED2016_17_PP.csv'
file_names = [fifteen, ten, five, current]

In [27]:
def import_all(fnames, schools_list) : #WILL NEED TO ADJUST THIS ONCE SCHOOLS LIST IS GONE
    #LOAD ALL SCHOOLS
    df = pd.DataFrame()
    fpath = 'C:/Users/jbachlombardo/Documents/Community Colleges/Data/College scorecard/CollegeScorecard_Raw_Data/'
    for n in fnames :
        year = re.findall('\d+_\d+', n)[0]
        f = fpath + n
        _ = pd.read_csv(f)
        _['Year'] = year
        _ = _[_['INSTNM'].isin(schools_list)]
        df = pd.concat([df, _])
    return df

In [55]:
def get_degree_changes(df) :
    """Create list of changes in degree grantings over set intervals, determined by entering fpath following College Scorecard filing naming convention and using degree list per CS data dictionary"""
    #LOAD LIST OF DEGREES
    with open(fpath + 'degrees_list.pkl', 'rb') as f:
        degrees = pickle.load(f)
    intervals_5yr = ['2001_02', '2006_07', '2011_12', '2016_17']
    #FILTER DOWN TO DEGREE LIST
    df = df.set_index(['Year', 'INSTNM'])[degrees].reset_index()
    changes_df = pd.DataFrame()
    for s in df.groupby('INSTNM') :
        t = s[1].T
        t.columns = list(t.iloc[0])
        t['15yr'] = t.iloc[2:][intervals_5yr[3]] - t.iloc[2:][intervals_5yr[0]]
        t['10yr'] = t.iloc[2:][intervals_5yr[3]] - t.iloc[2:][intervals_5yr[1]]
        t['05yr'] = t.iloc[2:][intervals_5yr[3]] - t.iloc[2:][intervals_5yr[2]]
        changes = {}
        for d in degrees :
            changes[d + '_05yr'] = t.loc[d, '05yr']
            changes[d + '_10yr'] = t.loc[d, '10yr']
            changes[d + '_15yr'] = t.loc[d, '15yr']
        _ = pd.DataFrame(data = changes, index = [t.loc['INSTNM', intervals_5yr[3]]])
        changes_df = pd.concat([changes_df, _])
    return changes_df

In [67]:
def get_degree_creation_removal(df) :
    data_dict = pd.read_excel('C:/Users/jbachlombardo/Documents/Community Colleges/Data/CC_datadictionary.xlsx')
    degs = list(data_dict['Name'].values)
    intervals_5yr = ['2001_02', '2006_07', '2011_12', '2016_17']
    school_names = list(df['INSTNM'].unique())
    df = df.set_index(['INSTNM', 'Year'])[degs]
    deg_create_5yr = list()
    deg_remove_5yr = list()
    deg_create_10yr = list()
    deg_remove_10yr = list()
    deg_create_15yr = list()
    deg_remove_15yr = list()
    for s in school_names :
        five = df.loc[(s, intervals_5yr[3])] - df.loc[(s, intervals_5yr[2])]
        deg_create_5yr.append(np.sum(five > 0))
        deg_remove_5yr.append(np.sum(five < 0))
        ten = df.loc[(s, intervals_5yr[3])] - df.loc[(s, intervals_5yr[1])]
        deg_create_10yr.append(np.sum(ten > 0))
        deg_remove_10yr.append(np.sum(ten < 0))
        fifteen = df.loc[(s, intervals_5yr[3])] - df.loc[(s, intervals_5yr[0])]
        deg_create_15yr.append(np.sum(fifteen > 0))
        deg_remove_15yr.append(np.sum(fifteen < 0))
    data = {'Schools': school_names, 'Deg_created_5yrs': deg_create_5yr, 'Deg_removed_5yr': deg_remove_5yr, 'Deg_created_10yrs': deg_create_10yr, 'Deg_removed_10yr': deg_remove_10yr, 'Deg_created_15yrs': deg_create_15yr, 'Deg_removed_15yr': deg_remove_15yr}
    create_remove_df = pd.DataFrame.from_dict(data = data).set_index('Schools')
    return create_remove_df

In [30]:
schools_test = ['Monroe Community College', 'Walla Walla Community College', 'Lake Area Technical Institute', 'MiraCosta College', 'Valencia College', 'Central Piedmont Community College', 'East Mississippi Community College']

In [31]:
df = import_all(file_names, schools_test)

C:\Users\jbachlombardo\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2903: DtypeWarning: Columns (1,1408,1431,1432,1433,1489,1537,1538,1539,1540,1542,1606,1610,1611,1614,1615,1616,1729) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\Users\jbachlombardo\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2903: DtypeWarning: Columns (6,9,1379,1380,1381,1382,1383,1384,1385,1386,1387,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1431,1432,1503,1504,1517,1518,1519,1529,1530,1531,1532,1534,1535,1537,1538,1539,1540,1542,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1606,1610,1611,1614,1615,1616,1708,1725,1726,1727,1728,1729,1743,1815,1816,1817,1818,1823,1824,1830,1831,1879,1880,1881,1882,1883,1884,1885,1886,1

In [56]:
pct_changes_df = get_degree_changes(df)

In [68]:
create_remove_df = get_degree_creation_removal(df)

In [77]:
current_1617 = pd.read_csv(fpath + file_names[3])

C:\Users\jbachlombardo\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (6,9,1379,1380,1381,1382,1383,1384,1385,1386,1387,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1431,1432,1503,1504,1517,1518,1519,1529,1530,1531,1532,1534,1535,1537,1538,1539,1540,1542,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1606,1610,1611,1614,1615,1616,1708,1725,1726,1727,1728,1729,1743,1815,1816,1817,1818,1823,1824,1830,1831,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [78]:
current_1617 = current_1617[current_1617['INSTNM'].isin(schools_test)]

In [80]:
current_1617_changes = current_1617.set_index('INSTNM').merge(pct_changes_df, left_index = True, right_index = True)

In [82]:
current_1617_changes = current_1617_changes.merge(create_remove_df, left_index = True, right_index = True)

In [83]:
current_1617_changes.to_csv(fpath + 'current_1617_changes_createremove.csv')

In [110]:
pct_change_cols = pct_changes_df.columns
create_remove_cols = create_remove_df.columns

In [111]:
data_dict_use = pd.read_excel('C:/Users/jbachlombardo/Documents/Community Colleges/Data/CC_datadictionary.xlsx', sheet_name = 'Use!')

In [117]:
data_dict_use = data_dict_use.dropna(subset = ['Label'])

In [126]:
current_cols = list(data_dict_use[data_dict_use['Set'] == 'CURRENT']['Label'].values)
cols_1516 = list(data_dict_use[data_dict_use['Set'] == '15-16']['Label'].values)
cols_1415 = list(data_dict_use[data_dict_use['Set'] == '14-15']['Label'].values)
cols_1314 = list(data_dict_use[data_dict_use['Set'] == '13-14']['Label'].values)
cols_1112 = list(data_dict_use[data_dict_use['Set'] == '11-12']['Label'].values)

In [132]:
data_dict_filters = pd.read_excel('C:/Users/jbachlombardo/Documents/Community Colleges/Data/CC_datadictionary.xlsx', sheet_name = 'Filters')

In [133]:
data_dict_filters = data_dict_filters.dropna(subset = ['Label'])

In [136]:
current_cols_filt = list(data_dict_filters[data_dict_filters['Set'] == 'CURRENT']['Label'].values)

In [128]:
current_cols.extend(pct_change_cols)

In [130]:
current_cols.extend(create_remove_cols)

In [137]:
current_cols.extend(current_cols_filt)

In [153]:
current_cols.remove('INSTNM')

In [138]:
cols_1516.append('UG25ABV')

In [139]:
inds_1516 = pd.read_csv(fpath + 'MERGED2015_16_PP.csv')
inds_1516 = inds_1516[inds_1516['INSTNM'].isin(schools_test)]

C:\Users\jbachlombardo\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1729) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [140]:
inds_1415 = pd.read_csv(fpath + 'MERGED2014_15_PP.csv')
inds_1415 = inds_1415[inds_1415['INSTNM'].isin(schools_test)]

C:\Users\jbachlombardo\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (6,1169,1170,1171,1172,1173,1174,1175,1177,1178,1179,1180,1183,1184,1185,1186,1187,1188,1196,1199,1200,1209,1212,1213,1222,1223,1224,1225,1226,1227,1235,1236,1237,1238,1239,1240,1248,1251,1252,1253,1255,1257,1261,1264,1265,1266,1268,1270,1274,1275,1276,1277,1278,1279,1281,1287,1288,1289,1290,1291,1292,1294,1303,1304,1316,1317,1326,1327,1328,1329,1330,1331,1335,1339,1340,1341,1342,1343,1344,1348,1379,1380,1381,1382,1383,1384,1385,1386,1387,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1411,1426,1427,1475,1476,1479,1480,1483,1484,1487,1488,1489,1490,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500,1501,1502,1503,1517,1529,1530,1532,1537,1540,1541,1542,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1606,1609,1

In [141]:
inds_1314 = pd.read_csv(fpath + 'MERGED2013_14_PP.csv')
inds_1314 = inds_1314[inds_1314['INSTNM'].isin(schools_test)]

C:\Users\jbachlombardo\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1537,1540,1542,1547,1548,1549,1561,1575,1606,1614,1615,1729) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [142]:
inds_1112 = pd.read_csv(fpath + 'MERGED2011_12_PP.csv')
inds_1112 = inds_1112[inds_1112['INSTNM'].isin(schools_test)]

In [164]:
current_merge = current_1617_changes[current_cols]
inds1516_merge = inds_1516.set_index('INSTNM')[cols_1516]
inds1415_merge = inds_1415.set_index('INSTNM')[cols_1415]
inds1314_merge = inds_1314.set_index('INSTNM')[cols_1314]
inds1112_merge = inds_1112.set_index('INSTNM')[cols_1112]

In [168]:
merged = current_merge.merge(inds1516_merge, left_index = True, right_index = True).merge(inds1415_merge, left_index = True, right_index = True).merge(inds1314_merge, left_index = True, right_index = True).merge(inds1112_merge, left_index = True, right_index = True)

In [175]:
merged.to_csv(fpath + 'merged_all_scorecard_indicators.csv')

In [211]:
merged = pd.read_csv(fpath + 'merged_all_scorecard_indicators_minus_longitudinal.csv')

In [214]:
merged = merged.replace('PrivacySuppressed', np.nan)

In [223]:
pcip_nulls = merged[merged['CIP42CERT1'].isnull()]['UNITID'].values

In [235]:
test = pd.read_csv(fpath  + 'MERGED2014_15_PP.csv')

C:\Users\jbachlombardo\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (6,1169,1170,1171,1172,1173,1174,1175,1177,1178,1179,1180,1183,1184,1185,1186,1187,1188,1196,1199,1200,1209,1212,1213,1222,1223,1224,1225,1226,1227,1235,1236,1237,1238,1239,1240,1248,1251,1252,1253,1255,1257,1261,1264,1265,1266,1268,1270,1274,1275,1276,1277,1278,1279,1281,1287,1288,1289,1290,1291,1292,1294,1303,1304,1316,1317,1326,1327,1328,1329,1330,1331,1335,1339,1340,1341,1342,1343,1344,1348,1379,1380,1381,1382,1383,1384,1385,1386,1387,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1411,1426,1427,1475,1476,1479,1480,1483,1484,1487,1488,1489,1490,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500,1501,1502,1503,1517,1529,1530,1532,1537,1540,1541,1542,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1606,1609,1

In [236]:
#test = test[((test['CONTROL'] == 1)) & ((test['HIGHDEG'] == 2) | (test['HIGHDEG'] == 1) | (test['HIGHDEG'] == 0))]
test[test['UNITID'].isin(pcip_nulls)][merged.columns[2:38]]

,PCIP01,PCIP03,PCIP04,PCIP05,PCIP09,PCIP10,PCIP11,PCIP12,PCIP13,PCIP14,...,PCIP42,PCIP43,PCIP44,PCIP45,PCIP46,PCIP47,PCIP48,PCIP49,PCIP50,PCIP51
7282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
